<a href="https://colab.research.google.com/github/ParvezAlam-AI/AI-App-Challenge-2024/blob/main/Stock_Assistanct_PA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modules

## pip install

In [1]:
!pip install gradio
!pip install yfinance
!pip install plotly
!pip install pandas
!pip install numpy
!pip install matplotlib

!pip insall requests
!pip install openai
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 915.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the 

## import

In [14]:
from google.colab import userdata
import pandas as pd
import openai
import gradio as gr
import yfinance as yf
import json
import os
import matplotlib.pyplot as plt
import io
from PIL import Image
import plotly.graph_objs as go  # Import Plotly for interactive charts
import re
import datetime

openai.api_key = userdata.get('apopen')

# code

## All the yfinance functions
1. format_stock_report
2. get_full_stock_report
3. get_stock_performance
4. get_recent_news
5. get_financial_metrics
6. get_company_overview


In [15]:


# Helper function to safely get data from dictionaries
def safe_get(data, key, default="-"):
    return data.get(key) if data.get(key) is not None else default

# Function to get company overview
def get_company_overview(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    overview = {
        "company_name": safe_get(info, "longName"),
        "ticker": safe_get(info, "symbol"),
        "exchange": safe_get(info, "exchange"),
        "industry": safe_get(info, "industry"),
        "ceo": safe_get(info, "ceo") or (info.get("companyOfficers")[0]['name'] if info.get("companyOfficers") else "N/A"),
        "year_founded": safe_get(info, "startDate"),
        "headquarters": f"{safe_get(info, 'city')}, {safe_get(info, 'state')}" if info.get("city") and info.get("state") else "N/A",
        "description": safe_get(info, "longBusinessSummary")
    }
    return overview


# Function to get financial metrics
def get_financial_metrics(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    metrics = {
        "market_cap": safe_get(info, "marketCap"),
        "total_revenue": safe_get(info, "totalRevenue"),
        "gross_profit_margin": safe_get(info, "grossMargins"),
        "ebitda_margin": safe_get(info, "ebitdaMargins"),
        "operating_margin": safe_get(info, "operatingMargins"),
        "net_profit_margin": safe_get(info, "profitMargins"),
        "eps_diluted": safe_get(info, "trailingEps"),
        "pe_ratio": safe_get(info, "trailingPE"),
        "forward_pe_ratio": safe_get(info, "forwardPE")
    }
    return metrics



def get_recent_news(ticker):
    stock = yf.Ticker(ticker)
    news_items = stock.news[:5]  # Get the latest 5 news articles
    news_list = []
    for item in news_items:
        news_list.append({
            "title": item.get("title"),
            "publisher": item.get("publisher"),
            "link": item.get("link"),
            "published_time": item.get("providerPublishTime")
        })
    return news_list




def get_stock_performance(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="10y")

    # Use .iloc for position-based indexing
    current_price = history['Close'].iloc[-1]
    fifty_two_week_low = safe_get(info, "fiftyTwoWeekLow")
    fifty_two_week_high = safe_get(info, "fiftyTwoWeekHigh")

    # Calculate returns using .iloc instead of direct indexing
    ytd_return = ((current_price - history['Close'].loc[history.index >= f"{pd.Timestamp.now().year}-01-01"].iloc[0]) / history['Close'].loc[history.index >= f"{pd.Timestamp.now().year}-01-01"].iloc[0]) * 100
    one_year_return = ((current_price - history['Close'].iloc[-252]) / history['Close'].iloc[-252]) * 100
    five_year_return = ((current_price - history['Close'].iloc[-1260]) / history['Close'].iloc[-1260]) * 100
    ten_year_return = ((current_price - history['Close'].iloc[0]) / history['Close'].iloc[0]) * 100

    performance = {
        "current_price": current_price,
        "52_week_range": f"{fifty_two_week_low} - {fifty_two_week_high}",
        "ytd_return": f"{ytd_return:.2f}%",
        "1y_total_return": f"{one_year_return:.2f}%",
        "5y_total_return_cagr": f"{(five_year_return/5):.2f}%",
        "10y_total_return_cagr": f"{(ten_year_return/10):.2f}%"
    }
    return performance





def get_full_stock_report(ticker):
    report = {
        "overview": get_company_overview(ticker),
        "financial_metrics": get_financial_metrics(ticker),
        "recent_news": get_recent_news(ticker),
        "stock_performance": get_stock_performance(ticker)
    }
    return report






# Function to format the stock report
def format_stock_report(report):
    overview = report.get('overview', {})
    financials = report.get('financial_metrics', {})
    news = report.get('recent_news', [])
    performance = report.get('stock_performance', {})

    formatted_report = f"""
**{overview.get('company_name')} ({overview.get('ticker')}) Overview**
- **Company Name:** {overview.get('company_name')}
- **Ticker:** {overview.get('ticker')}
- **Exchange:** {overview.get('exchange')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Year Founded:** {overview.get('year_founded')}
- **Headquarters:** {overview.get('headquarters')}

**Description:** {overview.get('description')}

**Financial Metrics & Fundamentals**
- **Market Cap:** {financials.get('market_cap')}
- **Total Revenues:** {financials.get('total_revenue')}
- **Gross Profit Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}
- **EPS Diluted:** {financials.get('eps_diluted')}
- **P/E Ratio:** {financials.get('pe_ratio')}
- **Forward P/E Ratio:** {financials.get('forward_pe_ratio')}

**Recent News**
"""
    for item in news:
        formatted_report += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')})\n"

    formatted_report += f"""
**Stock Performance**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Total Return:** {performance.get('ytd_return')}
- **1Y Total Return:** {performance.get('1y_total_return')}
- **5Y Total Return CAGR:** {performance.get('5y_total_return_cagr')}
- **10Y Total Return CAGR:** {performance.get('10y_total_return_cagr')}

**Summary**
*Provide a brief summary of the company's performance and outlook.*
"""
    return formatted_report


In [9]:
format_stock_report(get_full_stock_report('AAPL'))

"\n**Apple Inc. (AAPL) Overview**\n- **Company Name:** Apple Inc.\n- **Ticker:** AAPL\n- **Exchange:** NMS\n- **Industry:** Consumer Electronics\n- **CEO:** -\n- **Year Founded:** -\n- **Headquarters:** Cupertino, CA\n\n**Description:** Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, w

## Function

In [16]:

# Function to get ticker based on company name
def get_ticker_from_company_name(company_name):
    # This is a placeholder function to get a ticker from the company name
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=0,
        max_tokens=50,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0,
        messages=[
            {"role": "system", "content": "You are a stock information assistant. Provide the stock ticker for the given company name."},
            {"role": "user", "content": company_name}
        ]
    )

    try:
        ticker = response['choices'][0]['message']['content'].strip()  # Extract the ticker symbol
        return ticker
    except:
        return None  # If no ticker is found

In [19]:

# THIS IS 3RD CHANGE IN MAIN FUNCTION. Use OpenAI's Natural Language Processing: We can use OpenAI to determine the user’s intent.
# THIS IS FIRST TIME I HAVE LEARNED OPENAIS NLP TO DETERMIN USERS INTENT
# The model can be used to figure out whether the user wants just the stock price, a specific piece of financial data, or a full report.

def stock_chat(user_message):
    # Initial system message to define the bot's role
    messages = [
        {"role": "system", "content": "You are a stock market financial smart bot. You can provide stock related relavant information  prices, financial, trading, stock data, or full stock reports based on the user's query."},
        {"role": "user", "content": user_message}
    ]

    # Send user message to OpenAI to analyze intent
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=0,
        max_tokens=50,
        top_p=0.5,
        frequency_penalty=0,
        presence_penalty=0,
        messages=messages
    )

    try:
        # Extract the intent of the user query from the OpenAI response
        intent = response['choices'][0]['message']['content'].strip().lower()

        # Example intent detection logic:
        if "price" in intent:
            # User is asking for the stock price
            ticker = get_ticker_from_company_name(user_message)  # Extract the ticker based on the user query
            if ticker:
                stock = yf.Ticker(ticker)
                current_price = stock.history(period="1d")['Close'].iloc[-1]
                return f"The current stock price of {ticker} is ${current_price:.2f}."
            else:
                return f"It seems like you might be asking about the performance of a company named {ticker} today. However, I need more specific information to proceed, such as the full name or ticker symbol of the company."

        elif "financials" in intent or "metrics" in intent:
            # User is asking for specific financial metrics
            ticker = get_ticker_from_company_name(user_message)
            if ticker:
                financials = get_financial_metrics(ticker)
                return f"Here are the financial metrics for {ticker}: {financials}"
            else:
                return f"It seems like you might be asking about the performance of a company named {ticker} today. However, I need more specific information to proceed, such as the full name or ticker symbol of the company."

        elif "full report" in intent or "report" in intent:
            # User is asking for the full stock report
            ticker = get_ticker_from_company_name(user_message)
            if ticker:
                report = get_full_stock_report(ticker)
                formatted_report = format_stock_report(report)
                return formatted_report
            else:
                return f"It seems like you might be asking about the performance of a company named {ticker} today. However, I need more specific information to proceed, such as the full name or ticker symbol of the company."

        else:
            # If intent is not clear, fall back to this generic response
            return f"If you are referring to Paramount Global, the ticker symbol is {ticker}. Please confirm or provide the correct details so I can retrieve the relevant data for you."

    except Exception as e:
        return f"Error occurred: {str(e)}"

In [20]:
stock_chat("What is the current price of Tesla?")

ERROR:yfinance:$THE STOCK TICKER FOR TESLA IS TSLA.: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


'Error occurred: single positional indexer is out-of-bounds'

In [24]:


# Function to get ticker from user conversation
def get_ticker_from_conversation(conversation):
    # Get the last user message
    user_message = conversation[-1]['content']
    # Use OpenAI to extract the ticker symbol
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant that extracts stock ticker symbols from user messages."},
            {"role": "user", "content": user_message}
        ],
        max_tokens=10,
        temperature=0
    )
    ticker = response['choices'][0]['message']['content'].strip().upper()
    # Validate the ticker symbol format (1-5 uppercase letters)
    if re.match(r'^[A-Z]{1,5}$', ticker):
        return ticker
    else:
        return None

# Define function schemas for OpenAI's function calling
functions = [
    {
        "name": "get_full_stock_report",
        "description": "Retrieve a full stock report for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol, e.g., 'AAPL' for Apple Inc."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_company_overview",
        "description": "Get an overview of the company for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_financial_metrics",
        "description": "Get financial metrics for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_recent_news",
        "description": "Get recent news for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol."
                }
            },
            "required": ["ticker"]
        }
    },
    {
        "name": "get_stock_performance",
        "description": "Get stock performance metrics for a given ticker symbol.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "The stock ticker symbol."
                }
            },
            "required": ["ticker"]
        }
    }
]

# Function to handle user queries and interact with OpenAI
def stock_chat(conversation):
    # Define the system prompt
    system_prompt = """
You are a financial assistant that provides stock information. Users might refer to companies by name or ticker symbol. When necessary, determine the correct ticker symbol from the company name. Provide responses that mix numerical data and narrative descriptions.
"""
    # Prepare the messages
    messages = [{"role": "system", "content": system_prompt}] + conversation

    # Send the conversation to OpenAI
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # Use a model that supports function calling
        messages=messages,
        functions=functions,
        function_call="auto"
    )

    message = response['choices'][0]['message']

    # Check if the assistant wants to call a function
    if message.get("function_call"):
        function_name = message["function_call"]["name"]
        function_args = json.loads(message["function_call"]["arguments"])

        # Retrieve the ticker if not provided
        if not function_args.get("ticker"):
            ticker = get_ticker_from_conversation(conversation)
            if ticker:
                function_args["ticker"] = ticker
            else:
                return "I'm sorry, I couldn't determine the ticker symbol for the company you're referring to. Please provide the ticker symbol."

        ticker = function_args["ticker"]

        # Dynamically call the appropriate function
        if function_name == "get_full_stock_report":
            report = get_full_stock_report(ticker)
            formatted_report = format_stock_report(report)
            # Append the function response to the conversation
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_report
            })
            # Get the final assistant response
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            # Append to conversation and return
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_company_overview":
            overview = get_company_overview(ticker)
            # Format and return the overview
            formatted_overview = f"""
**Company Overview for {ticker}**
- **Company Name:** {overview.get('company_name')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Description:** {overview.get('description')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_overview
            })
            # Get the final assistant response
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_financial_metrics":
            metrics = get_financial_metrics(ticker)
            # Format and return the metrics
            formatted_metrics = f"""
**Financial Metrics for {ticker}**
- **Market Cap:** {metrics.get('market_cap')}
- **Total Revenue:** {metrics.get('total_revenue')}
- **Gross Profit Margin:** {metrics.get('gross_profit_margin')}
- **EPS Diluted:** {metrics.get('eps_diluted')}
- **P/E Ratio:** {metrics.get('pe_ratio')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_metrics
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_recent_news":
            news_items = get_recent_news(ticker)
            # Format and return the news
            formatted_news = f"**Recent News for {ticker}**\n"
            for item in news_items:
                formatted_news += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')})\n"
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_news
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        elif function_name == "get_stock_performance":
            performance = get_stock_performance(ticker)
            # Format and return the performance data
            formatted_performance = f"""
**Stock Performance for {ticker}**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Return:** {performance.get('ytd_return')}
- **1-Year Total Return:** {performance.get('1y_total_return')}
"""
            conversation.append({
                "role": "function",
                "name": function_name,
                "content": formatted_performance
            })
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=conversation
            )
            assistant_message = final_response['choices'][0]['message']['content']
            conversation.append({"role": "assistant", "content": assistant_message})
            return assistant_message

        else:
            return "I'm sorry, I couldn't process your request."
    else:
        # If no function call, return the assistant's message
        assistant_response = message.get("content")
        conversation.append({"role": "assistant", "content": assistant_response})
        return assistant_response

# Initialize conversation history
conversation_history = []

# Function to handle user input
def handle_user_input(user_input):
    # Append user message to conversation
    conversation_history.append({"role": "user", "content": user_input})
    # Get assistant response
    assistant_response = stock_chat(conversation_history)
    return assistant_response

# # Example usage
# if __name__ == "__main__":
#     while True:
#         user_query = input("You: ")
#         response = handle_user_input(user_query)
#         print(f"Assistant:\n{response}\n")


In [25]:
handle_user_input(" how is tesla trading today")

"As of the latest available data, Tesla (TSLA) is trading at a current price of $250.08. The stock has a 52-week trading range of $138.8 to $271.0. Year-to-date, Tesla has seen a return of 0.67%, and the stock has a one-year total return of -3.83%. It's important to note that stock prices can fluctuate throughout the trading day, so it's advisable to check for real-time updates for the most current information."

In [26]:
handle_user_input("which is better netflix or tesla")

'Based on the provided information, here is a comparison between Netflix (NFLX) and Tesla (TSLA):\n\n**Netflix (NFLX):**\n- Netflix is a leading entertainment service provider, offering a wide range of content to subscribers worldwide.\n- The stock has seen significant growth, with a YTD total return of 53.62% and a 1-year total return of 93.16%.\n- Netflix has demonstrated strong financial metrics, with a healthy gross profit margin of 43.84% and a net profit margin of 19.54%.\n\n**Tesla (TSLA):**\n- Tesla is a prominent electric vehicle company known for its innovative approach to sustainable transportation.\n- The stock has a YTD return of 0.67% and a 1-year total return of -3.83%.\n- Tesla\'s P/E ratio stands at 45.04, indicating it may be considered relatively high compared to Netflix\'s forward P/E ratio of 31.28.\n\nIn terms of stock performance and financial metrics, Netflix has shown stronger growth and returns compared to Tesla. However, the decision on which stock is "better

In [27]:
handle_user_input("what did etsy trading says for this week")

"As of the latest available data, Etsy (ETSY) is trading at a current price of $52.80. The stock has a 52-week trading range of $50.55 to $89.58. Year-to-date, Etsy has seen a return of -34.88%, and the stock has a one-year total return of -16.00%. It's important to note that stock prices can vary throughout the week, so it's advisable to monitor real-time updates for the most current information on Etsy's trading performance."